In [2]:
%pip install influxdb_client

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
from influxdb_client import InfluxDBClient
from datetime import datetime, timedelta

# 디바이스 ID와 위치를 매핑
location_mapping = {
    '24e124126d152919': 'indoor',
    '24e124126d152969': 'bottom_right_corner',
    '24e124128c067999': 'indoor',
    '24e124785c389818': 'bottom_left_corner',
    '24e124785c421885': 'top_right_corner'
}

# InfluxDB 설정 정보
url = "http://133.186.144.22:8086"
token = "BPJ1pnKvoaov4Tte971t0zpRSTUXNZvrshU7u3UPheAIsBeUJEFfbKjfsZjtwZmugkHJEGRW17lH4bR9ybanNQ=="
org = "smoothing"

# InfluxDB 클라이언트 생성
def create_client(url, token, org):
    return InfluxDBClient(url=url, token=token, org=org)

# 쿼리 실행 및 DataFrame으로 변환
def query_to_dataframe(client, query):
    result = client.query_api().query(query=query)
    results = []
    
    for table in result:
        for record in table.records:
            results.append({
                "time": record.get_time(),
                "value": record.get_value(),
                "place": record.values.get("place"),
                "location": record.values.get("location"),
                "device": record.values.get("device")
            })
    
    df = pd.DataFrame(results)
    df['time'] = df['time'].astype(str).str.replace(r'\+00:00$', '', regex=True)
    return df

# 데이터를 날짜를 지정하여 CSV 파일로 저장
def save_csv(df, file_pattern, directory):
    # 경로가 존재하는지 확인하고, 없다면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    current_date = datetime.now()
    previous_date = current_date - timedelta(days=1)
    filename = f"{directory}{previous_date.strftime(file_pattern)}"
    df.to_csv(filename, index=False)
    
# 온도 Data에서 'device' 열에 따라 'location' 열을 업데이트    
def update_location(df, location_mapping):
    df['location'] = df['device'].map(location_mapping)
    return df

### 쿼리 사용하여 조회

In [9]:
# 클라이언트 생성 및 쿼리 실행
client = create_client(url, token, org)

# 전력 조회 Flux 쿼리
query_power_data = '''
from(bucket: "powermetrics_data")
  |> range(start: 2024-04-19T00:58:00Z, stop: 2024-04-19T02:00:00Z)
  |> filter(fn: (r) => r["phase"] == "total")
  |> filter(fn: (r) => r["location"] == "air_conditioner")
  |> filter(fn: (r) => r["description"] == "w")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# CSV 변환
df_power = query_to_dataframe(client, query_power_data)
print(df_power.head())
save_csv(df_power, "04_26_power_data.csv", "f/")

# # 온도 조회 Flux 쿼리
# query_temperature_data = '''
# import "date"
# from(bucket: "environmentalsensors_data")
#   |> range(start: 2024-04-26T00:00:00Z, stop: 2024-04-27T00:00:00Z)
#   |> filter(fn: (r) => r["measurement"] == "temperature")
#   |> filter(fn: (r) => r["place"] == "class_a" or r["place"] == "office")
#   |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
#   |> keep(columns: ["_time", "_value", "place", "location", "device"])
# '''
# 
# # CSV 변환
# df_temperature = query_to_dataframe(client, query_temperature_data)
# df_temperature_fix = update_location(df_temperature, location_mapping)
# print(df_temperature_fix.head())
# save_csv(df_temperature_fix, "04_26_temperature_data.csv", "temperature/")
# 
# # 클라이언트 종료
# client.close()


                  time  value   place         location     device
0  2024-04-19 01:00:00  946.0  office  air_conditioner  gems-3500
1  2024-04-19 01:02:00  949.0  office  air_conditioner  gems-3500
2  2024-04-19 01:04:00  947.5  office  air_conditioner  gems-3500
3  2024-04-19 01:06:00  948.0  office  air_conditioner  gems-3500
4  2024-04-19 01:08:00  948.0  office  air_conditioner  gems-3500


In [20]:
# 클라이언트 생성 및 쿼리 실행
client = create_client(url, token, org)

# 이산화탄소 조회 Flux 쿼리
query_co2_data = '''
import "date"
from(bucket: "environmentalsensors_data")
  |> range(start: 2024-04-27T00:00:00Z, stop: 2024-04-28T00:00:00Z)
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["measurement"] == "co2")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# CSV 변환
df_co2 = query_to_dataframe(client, query_co2_data)
df_co2_fix = update_location(df_co2, location_mapping)
print(df_co2_fix.head())
save_csv(df_co2_fix, "04_27_co2_data.csv", "co2/")

# 클라이언트 종료
client.close()

                  time  value    place location            device
0  2024-04-27 00:02:00  669.0  class_a   indoor  24e124128c067999
1  2024-04-27 00:04:00  674.5  class_a   indoor  24e124128c067999
2  2024-04-27 00:06:00  684.0  class_a   indoor  24e124128c067999
3  2024-04-27 00:08:00  683.5  class_a   indoor  24e124128c067999
4  2024-04-27 00:10:00  670.0  class_a   indoor  24e124128c067999
